# Generating [Robot][] Blocks for [Blockly][]

WHile Robot Frameworks's natural language approach to writing code is very approachable, it still suffers from a _discoverability_ problem, where a user _doesn't know what they don't know_:

> _What's a keyword? A Test Case? A Test Suite? A Variable?_

> _What settings can I use and where?_

> _What are the arguments to a keyword, and what do they mean?_

> _How do I run these things?_

Blockly provides some answers in the form of _blocks_ which a user can **directly manipulate** to generate new programs, data or other procedural structures.

While the first two questions require special handling to generate valid syntax, Robot's [libdoc][] provides a wealth of information about the Hard Part, which is the discovery of keywords.

Further, JupyterLab and `robotkernel` close the loop, and allow for introspection and execution, as well as handling file saving.


[robot]: https://github.com/robotframework/robotframework
[blockly]: http://github.com/blockly/blockly
[libdoc]: https://github.com/robotframework/robotframework/blob/master/src/robot/libdocpkg/builder.py#L34

In [ ]:
from stringcase import snakecase
from urllib.parse import quote
from pathlib import Path
import json
from xml.etree import ElementTree as et
from robot.libdocpkg import LibraryDocumentation

Initially, let's use Robot's rich standard library to populate some keywords.

In [ ]:
from robot.libraries import STDLIBS

In [ ]:
HERE = Path(".")
BLOCKS = (HERE / ".." / "blocks")
TOOLBOX = (HERE / ".." / "xml")

(BLOCKS / "robot").mkdir(exist_ok=True, parents=True)

We'll be looking at both **Library** documentation:

In [ ]:
libdoc = LibraryDocumentation("BuiltIn")
libdoc

...as well as **Keyword** documentation:

In [ ]:
kwdoc = libdoc.keywords[0]
kwdoc

## What is a _&lt;Insert Robot Thing Here>_?
To show a concept to a user in Blockly, we'll build a **Toolbox**, which is just a (sigh) XML tree.
Moving a little fast and loose, we'll reuse an existing Toolbox which includes all of the most-commonly implemented blocks. Some of these will not be easy (or desirable) to implement in robot, but that's for another day.

While blockly can generate lots of languages, the reference UI implementation is implemented in JavaScript. Since we'd like this to be as data-driven as possible, we'll be exploring ways to encode critical information about blocks and their language-specific implementation by other means.

In [ ]:
all_blocks = {}

In [ ]:
toolbox = et.parse(str(TOOLBOX / "toolbox.xml")).getroot()

Our initial categories will be concerned with core language semantics. Behold, the python standardlib xml library (bleah).

In [ ]:
def eb(tag, parent=None, **attrib):
    el = et.Element(tag)
    el.attrib.update(attrib)
    if parent is not None:
        parent.append(el)
    return el

In [ ]:
tool_cat = eb('category', toolbox, name='Robot')
cat_blocks = all_blocks["Robot"] = dict()

### The Test Case block
While the new-fangled _Robot Process Automation_ will introduce **Tasks**, most existing Robot scripts are all about **Test Cases**. A _normal_ test case consists of:
- A (unique within a suite) name
- Some **Test Case Setting**s
- Some **Keyword**s
- One or more **Variable Assignment**s

The data structure for defining a block is... idiomatic. The [Block Factory](https://blockly-demo.appspot.com/static/demos/blockfactory) makes it slightly more bearable to build block definitions... but we're going to have to get a lot more automated to document the 300+ standard library keywords!

In [ ]:
def make_shadow_value(tool_block, name, default):
    value = eb("value", tool_block, name=name)
    shadow = eb("shadow", value, type="text")
    field = eb("field", shadow, name="TEXT")
    field.text = default

In [ ]:
def make_table_block(name, tooltip, shadow_name=True, **block):
    block_type = f"robot___{snakecase(name)}"
    tool_block = eb("block", tool_cat, type=block_type)
    _block = {
      "type": block_type,
      "message0": f"{name} %1{' %2' if shadow_name else ''}",
      "args0": [
        {"type": "input_value", "name": "NAME", "check": "String"},
        {"type": "input_statement", "name": "ROWS", "check": None}
      ],
      "tooltip": tooltip
    }
    
    if shadow_name:
        make_shadow_value(tool_block, "NAME", f"An Untitled {name}")

    if block:
        _block.update(**block)

    cat_blocks[block_type] = dict(
        name=name,
        args=[],
        block=_block,
    )

In [ ]:
make_table_block("Settings", "Things about all the tests",
    shadow_name=False,
    args0=[{"type": "input_statement", "name": "ROWS", "check": None}],
)
make_table_block("Test Case", "A thing you want to test")
make_table_block("Keyword", "A task a Robot can perform")

Settings are (mostly) consistent.

In [ ]:
def make_setting_block(name):
    """ this needs a lot of work: scopes, typing (some accept keywords, lists)
    """
    block_type = f"robot___setting___{snakecase(name)}"
    tool_block = eb("block", tool_cat, type=block_type)
    _block = {
        "type": block_type,
        "message0": f"{name} %1",
        "args0": [
            {"type": "input_value", "name": "VALUE", "check": None},
        ],
        "nextStatement": None,
        "previousStatement": None,
    }

    cat_blocks[block_type] = dict(
        name=name,
        args=[],
        block=_block,
    )

In [ ]:
list(map(make_setting_block, [
    "Documentation", 
    "Arguments", 
    "Return", 
    "Setup",
    "Teardown", 
    "Suite Setup",
    "Suite Teardown",
    "Test Setup",
    "Test Teardown",
    "Test Timeout",
    "Library",
    "Resource",
    "Variables",
    "Metadata",
    "Template",
    "Test Template"
    "Tags",
    "Force Tags",
    "Default Tags"
]))

In [ ]:
def make_lib_blocks(libdoc, toolbox, all_blocks):
    tool_cat = eb('category', toolbox, name=libdoc.name)

    snake_name = snakecase(lib_name)
    lib_blocks = all_blocks[lib_name] = {}
    
    for kwdoc in sorted(libdoc.keywords, key=lambda k: k.name):
        make_keyword_block(kwdoc, snake_name, tool_cat, lib_blocks)

In [ ]:
def make_keyword_block(kwdoc, snake_name, tool_cat, lib_blocks):
    block_type = f"robot_{snake_name}___{snakecase(kwdoc.name)}"

    tool_block = eb("block", tool_cat, type=block_type)

    block = dict(
        tooltip=kwdoc.shortdoc,
        helpUrl=f"http://robotframework.org/robotframework/latest/libraries/{lib_name}.html#{quote(kwdoc.name)}",
        type=block_type,
        message0=f"{kwdoc.name} %1",
        args0=[{"type": "input_dummy"}],
        nextStatement=None,
        previousStatement=None,
        output='Keyword',
    )

    args = []
    for i, arg in enumerate(kwdoc.args):
        if "*" in arg: continue
        if "=" in arg:
            arg, value = arg.split("=")
            make_shadow_value(tool_block, arg, value)

        block.update({
            f"message{i + 1}": f"{arg} %1",
            f"args{i + 1}": [{
                "type": "input_value",
                "name": arg
            }]
        })
        args.append(arg)

    lib_blocks[block_type] = dict(
        name=kwdoc.name,
        block=block,
        args=args,
        template=f"""    {kwdoc.name}""",
    )

In [ ]:
for lib_name in sorted(STDLIBS):
    try: libdoc = LibraryDocumentation(lib_name)
    except: continue
    make_lib_blocks(libdoc, toolbox, all_blocks)

In [ ]:
(BLOCKS / "robot" / "stdlibs.json").write_text(json.dumps(all_blocks, indent=2, sort_keys=True))
(TOOLBOX / "robot.xml").write_bytes(et.tostring(toolbox))